In [1]:
import zipfile
import numpy as np
import skimage.io
import skimage.viewer
import skimage.filters
import skimage.measure
import skimage.morphology
import matplotlib.pyplot as plt
import matplotlib
import cv2
from PIL import Image
import io
from plantcv import plantcv as pcv
import os
import glob

/home/myrthe/miniconda3/envs/plantcv/lib/python3.9/site-packages/skimage/viewer/utils/__init__.py:1: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  from .core import *


In [ ]:
# Import all time slots
archive_ts1 = zipfile.ZipFile('NDVI/20210521_FX10_NDVI.zip', 'r')
archive_ts2 = zipfile.ZipFile('NDVI/20210528_FX10_NDVI.zip', 'r')
archive_ts3 = zipfile.ZipFile('NDVI/20210601_FX10_NDVI.zip', 'r')
archive_ts4 = zipfile.ZipFile('NDVI/20210604_FX10_NDVI.zip', 'r')
archive_ts5 = zipfile.ZipFile('NDVI/20210608_FX10_NDVI.zip', 'r')
archive_ts6 = zipfile.ZipFile('NDVI/20210611_FX10_NDVI.zip', 'r')
archive_ts7 = zipfile.ZipFile('NDVI/20210615_FX10_NDVI.zip', 'r')
archive_ts8 = zipfile.ZipFile('NDVI/20210618_FX10_NDVI.zip', 'r')
archive_ts9 = zipfile.ZipFile('NDVI/20210622_FX10_NDVI.zip', 'r')
archive_ts10 = zipfile.ZipFile('NDVI/20210625_FX10_NDVI.zip', 'r')

In [ ]:
# Get the files in the archive
files_ts1 = archive_ts1.namelist()[1::]
files_ts2 = archive_ts2.namelist()
files_ts3 = archive_ts3.namelist()
files_ts4 = archive_ts4.namelist()
files_ts5 = archive_ts5.namelist()
files_ts6 = archive_ts6.namelist()
files_ts7 = archive_ts7.namelist()
files_ts8 = archive_ts8.namelist()
files_ts9 = archive_ts9.namelist()
files_ts10 = archive_ts10.namelist()

In [ ]:
import os
sativa_ILs = os.listdir("Sativa_AS") 

In [ ]:
sativa_ILs = [x[6:11] for x in sativa_ILs]

In [ ]:
def filter_sat(all_files):
    sat = []
    for f in all_files:
        for s in sativa_ILs:
            if s in f:
                sat.append(f)
    return sat

In [ ]:
files_ts1_sat = filter_sat(files_ts1)
files_ts2_sat = filter_sat(files_ts2)

In [ ]:
def thresholding(image_gray):
    a_thresh = pcv.threshold.otsu(gray_img=image_gray, max_value=255, object_type='dark')
    a_fill = pcv.fill(bin_img=a_thresh, size=200)
    dilated = pcv.dilate(gray_img=a_fill, ksize=2, i=1)
    Mask = cv2.bitwise_not(dilated)
    return Mask

In [ ]:
def observations(image, obj, mask):
    analysis_image = pcv.analyze_object(img=image, obj=obj, mask=mask, label="default")
    plant_area = pcv.outputs.observations
    cha = plant_area['default']['convex_hull_area']['value']
    ec = plant_area['default']['ellipse_center']['value']
    return cha, ec

In [ ]:
def resizing(gray_image_other, gray_image, rgb_image):
    h, w = gray_image_other.shape
    dim = (w, h)
    resized_g = cv2.resize(gray_image, dim, interpolation = cv2.INTER_AREA)
    resized_rgb = cv2.resize(rgb_image, dim, interpolation = cv2.INTER_AREA)
    return resized_g, resized_rgb

In [ ]:
import math
def gr_calc(ec_list_past, cha_list_past, ec_list_present, cha_list_present): 
    growth_rates = []
    for x_past, y_past in zip(ec_list_past, cha_list_past):
        dist_list = []
        close_past = []
        close_present = []
        size_past = []
        size_present = []
        for x_present, y_present in zip(ec_list_present, cha_list_present):
            distance = math.dist(x_past, x_present)
            dist_list.append(distance)
            close_past.append(x_past)
            close_present.append(x_present)
            size_past.append(y_past)
            size_present.append(y_present)
        min_value = min(dist_list)
        min_index = dist_list.index(min_value)
        growth_rates.append((size_present[min_index] - size_past[min_index]) / size_past[min_index])
    return growth_rates

In [ ]:
# Lactuca sativa
import gc
pcv.params.debug = None
def growth_rate(files_present, files_past, archive_present, archive_past):
    for file_present, file_past in zip(files_present, files_past):
        if file_present[18:23] == file_past[18:23]:
            imgdata_present = archive_present.read(file_present)
            imgdata_past = archive_past.read(file_past)
            bytes_present = io.BytesIO(imgdata_present)
            bytes_past = io.BytesIO(imgdata_past)
            
            #Present
            im_g_present = Image.open(bytes_present).convert('L')
            im_rgb_present = Image.open(bytes_present).convert('RGB')
            np_g_present = np.array(im_g_present) 
            np_rgb_present = np.array(im_rgb_present)
            mask_present = thresholding(np_g_present)
            obj_present, obj_hierarchy_present = pcv.find_objects(img=np_rgb_present, mask=mask_present)
            filter_obj_present=[x for x in obj_present if len(x)>=50]
            filter_obj_present = np.array(filter_obj_present)
            cha_list_present = []
            ec_list_present = []
            for o_p in filter_obj_present:
                cha_present, ec_present = observations(np_rgb_present, o_p, mask_present)
                cha_list_present.append(cha_present)
                ec_list_present.append(ec_present)
                
            
            #Past
            im_g_past = Image.open(bytes_past).convert('L')
            im_rgb_past = Image.open(bytes_past).convert('RGB')
            np_g_past = np.array(im_g_past) 
            np_rgb_past = np.array(im_rgb_past)
            resized_g, resized_rgb = resizing(np_g_present, np_g_past, np_rgb_past)
            mask_past = thresholding(resized_g)
            obj_past, obj_hierarchy_past = pcv.find_objects(img=resized_rgb, mask=mask_past)
            filter_obj_past =[x for x in obj_past if len(x)>=50]
            filter_obj_past = np.array(filter_obj_past)
            cha_list_past = []
            ec_list_past = []
            for o_pr in filter_obj_past:
                cha_past, ec_past = observations(resized_rgb, o_pr, mask_past)
                cha_list_past.append(cha_past)
                ec_list_past.append(ec_past)
                
            growth_rates = gr_calc(ec_list_past, cha_list_past, ec_list_present, cha_list_present)
            print(mean(growth_rates))
            gc.collect()
            #return growth_rates

In [ ]:
growth_rate(files_ts2_sat, files_ts1_sat, archive_ts2, archive_ts1)